In [1]:
"""
pip install opencv-python  # required for cv2
"""

import cv2
import numpy as np
from pathlib import Path
from glob import glob

In [10]:
def collate_images(background_path, image_paths, output_dir=Path("output"), scale_factor=0.8, prefix="", suffix="bgd"):
    """
    Collates images with a background, preserving aspect ratio and using a fixed scale factor.

    Args:
        background_path: Path to the background image.
        image_paths: List of paths to images to be collated.
        output_dir: Output directory Path for the collated images.
        scale_factor: The scaling factor for the images (0.0 - 1.0).
        suffix: added to the new output filename
    """
    # Ensure output directory exists
    output_dir.mkdir(parents=True, exist_ok=True)
    
    try:
        background = cv2.imread(str(background_path))
    except cv2.error as e:
        print(f"Error reading background image: {e}")
        return False

    bg_height, bg_width, _ = background.shape

    ok_images = 0
    for image_path in image_paths:
        print(f"Collating {image_path} ...")
        try:
            image = cv2.imread(str(image_path))
        except cv2.error as e:
            print(f"Error reading image: {image_path}, {e}")
            continue  # Skip to the next image            

        if image is None: 
            continue
            
        img_height, img_width, _ = image.shape
        
        # Check if dimensions are compatible
        if img_width > bg_width or img_height > bg_height:
            # Skip image if dimensions are incompatible
            print(f"\t[WARNING] Image dimensions are incompatible: {image_path}")
            continue        

        try:
            # Determine the larger dimension and calculate new size based on scale factor
            if img_width > img_height:
                new_width = int(bg_width * scale_factor)
                new_height = int(img_height * (new_width / img_width))
            else:
                new_height = int(bg_height * scale_factor)
                new_width = int(img_width * (new_height / img_height))

            # Resize image
            resized_image = cv2.resize(image, (new_width, new_height))

            # Create a mask for the image
            mask = np.zeros_like(resized_image)
            mask[0:new_height, 0:new_width] = 255

            # Calculate position for placing the image on the background
            x = (bg_width - new_width) // 2
            y = (bg_height - new_height) // 2

            # Create a copy of the background image
            result = background.copy()

            # Put the resized image on the background
            result[y:y+new_height, x:x+new_width] = resized_image

            # Save the resulting image
            if prefix:
                new_filename = f"{prefix}__" + Path(image_path).stem + f"-{suffix}" + Path(image_path).suffix
            else:
                new_filename = Path(image_path).stem + f"-{suffix}" + Path(image_path).suffix
            output_path = output_dir / Path(new_filename)
            print(f"\tOutput file: {output_path}")
            cv2.imwrite(str(output_path), result)
            ok_images += 1
        except Exception as e:
            print(f"\t[ERROR] Failed processing '{image_path}'\n {e}")
            continue  # Skip to the next image    
            
    return len(image_paths), ok_images

In [11]:
background_path = "00-background-stary-universe.png"

In [12]:
ROOT_DIR = "3-A-Beautiful-Soul"
ITER_DIR = "iter-2"

In [13]:
excluded_images = [
    'archive',
]
excluded_images = [f"{ROOT_DIR}\\{x}" for x in excluded_images]

In [14]:
source_images = glob(f"{ROOT_DIR}/original/*")
image_paths = [x for x in source_images if "-bgd" not in x]
# minimum border
total_images, ok_images = collate_images(Path(background_path), image_paths, output_dir=Path(f"{ROOT_DIR}/{ITER_DIR}"), scale_factor=0.98)
print(f"input_images, total_images, ok_images = {len(source_images)}, {total_images}, {ok_images}")

Collating 3-A-Beautiful-Soul/original\02-candle-light-2024-08-04.png ...
	Output file: 3-A-Beautiful-Soul\iter-2\02-candle-light-2024-08-04-bgd.png
Collating 3-A-Beautiful-Soul/original\03-01-UChicago-Campus.jpg ...
	Output file: 3-A-Beautiful-Soul\iter-2\03-01-UChicago-Campus-bgd.jpg
Collating 3-A-Beautiful-Soul/original\03-sf-golden-gate-bridge.png ...
	Output file: 3-A-Beautiful-Soul\iter-2\03-sf-golden-gate-bridge-bgd.png
Collating 3-A-Beautiful-Soul/original\04-pacific-ocean-01.png ...
	Output file: 3-A-Beautiful-Soul\iter-2\04-pacific-ocean-01-bgd.png
Collating 3-A-Beautiful-Soul/original\05-sun-rise-ocean.jpg ...
	Output file: 3-A-Beautiful-Soul\iter-2\05-sun-rise-ocean-bgd.jpg
Collating 3-A-Beautiful-Soul/original\06-tdlee-institute.png ...
	Output file: 3-A-Beautiful-Soul\iter-2\06-tdlee-institute-bgd.png
Collating 3-A-Beautiful-Soul/original\07-tdlee-library.png ...
	Output file: 3-A-Beautiful-Soul\iter-2\07-tdlee-library-bgd.png
Collating 3-A-Beautiful-Soul/original\08-House

### Fix manually
```
[ERROR] Failed processing '3-A-Beautiful-Soul/original\10-Taihu_Lake,_Wuxi_20180428.jpg'
[ERROR] Failed processing '3-A-Beautiful-Soul/original\11-Sunset_at_taihu_wuxi.jpg'
[ERROR] Failed processing '3-A-Beautiful-Soul/original\13-huang-shan-4.jpg'
[ERROR] Failed processing '3-A-Beautiful-Soul/original\17-1024px-Nasa_earth.jpg'
[ERROR] Failed processing '3-A-Beautiful-Soul/original\18-total-solar-eclipse.jpg'
[ERROR] Failed processing '3-A-Beautiful-Soul/original\20-galaxy-01.png'
[ERROR] Failed processing '3-A-Beautiful-Soul/original\21-universe.jpg'
```

### Compare iter-2 vs iter-1

In [21]:
iter1_images = glob(f"{ROOT_DIR}/iter-1/*")
iter2_images = glob(f"{ROOT_DIR}/iter-2/*")

iter1_images = [x.replace("/iter-1", "").replace(".jpg", "").replace(".png", "") for x in iter1_images]
iter2_images = [x.replace("/iter-2", "").replace(".jpg", "").replace(".png", "") for x in iter2_images]

len(iter1_images), len(iter2_images)

(26, 26)

In [22]:
set(iter1_images).difference(set(iter2_images))

set()

In [23]:
set(iter2_images).difference(set(iter1_images))

set()